In [2]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# 1. Cargar y preparar los datos
data = pd.read_csv('DatosSingapore.csv')  # Reemplazar con el archivo limpio
data.dropna(inplace=True)  # Asegurarnos de no tener datos faltantes

# Separar características (X) y variable objetivo (y)
X = data.drop(columns=['price'])  # 'price' es la variable que queremos predecir
y = data['price']

# Codificar variables categóricas
X = pd.get_dummies(X, drop_first=True)

print(data.head())

       id  host_id  host_response_rate  host_acceptance_rate  \
0   71896   367042                 1.0                   1.0   
1   71903   367042                 1.0                   1.0   
2  294281  1521514                 1.0                   1.0   
3  344803   367042                 1.0                   1.0   
4  369141  1521514                 1.0                   1.0   

   host_is_superhost  host_total_listings_count  host_has_profile_pic  \
0              False                         10                  True   
1              False                         10                  True   
2              False                          8                  True   
3              False                         10                  True   
4              False                          8                  True   

   host_identity_verified  latitude  longitude  ...  \
0                    True   1.34754  103.95958  ...   
1                    True   1.34531  103.96100  ...   
2          

In [4]:
# Crear la variable objetivo para clasificación
data['recommended'] = np.where(
    (data['price'] <= 200) & 
    (data['review_scores_rating'] >= 4.5) & 
    (data['room_type'] == 'Entire home/apt'), 1, 0
)

# Separar características (X) y variable objetivo (y)
X = data.drop(columns=['recommended', 'price'])
y = data['recommended']

# Codificar variables categóricas
X = pd.get_dummies(X, drop_first=True)

# Normalizar las variables numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Crear el modelo de clasificación
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Activación sigmoid para clasificación binaria
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluar el modelo
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype("int32")

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_classes))

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Recomendado', 'Recomendado'], yticklabels=['No Recomendado', 'Recomendado'])
plt.xlabel('Predicción')
plt.ylabel('Valor real')
plt.title('Matriz de confusión')
plt.show()

KeyError: 'room_type'